In [0]:
dbutils.fs.ls("/FileStore/tables/")


Out[2]: [FileInfo(path='dbfs:/FileStore/tables/Players.csv', name='Players.csv', size=281111, modificationTime=1661339704000),
 FileInfo(path='dbfs:/FileStore/tables/Seasons_Stats.csv', name='Seasons_Stats.csv', size=5117407, modificationTime=1661339729000),
 FileInfo(path='dbfs:/FileStore/tables/player_data.csv', name='player_data.csv', size=346695, modificationTime=1661339704000),
 FileInfo(path='dbfs:/FileStore/tables/players.csv', name='players.csv', size=2404263, modificationTime=1661359310000),
 FileInfo(path='dbfs:/FileStore/tables/teams.csv', name='teams.csv', size=67791, modificationTime=1661359299000)]

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
spark.version

Out[11]: '3.2.1'

## NBA igraci sa osnovnim informacijama od 1950 do 2021

U ovom datasetu se nalaze osnovne informacije o NBA igracima koji su u jednom periodu bili aktivni igraci, od 1950. pa sve do 2021. godine

Tabela se sastoji iz sledecih kolona:
- **name**: ime i prezime igraca
- **year_start**: prva aktivna godina igraca u NBA ligi
- **year_end**: poslednja aktivna godina igraca u NBA ligi
- **position**: pozicija igraca
- **height**: visina igraca
- **weight**: tezina igraca
- **birth_date**: datum rodjenja
- **college**: univerzitet koji je igrac pohadjao

izvor: https://www.kaggle.com/datasets/drgilermo/nba-players-stats?select=player_data.csv //Kaggle

In [0]:
file_location = "/FileStore/tables/player_data.csv"
df1 = spark.read.option("header","true").option("inferSchema","true").csv(file_location)

df1.printSchema()
df1.show(30,truncate = False)

root
 |-- name: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_end: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- college: string (nullable = true)

+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+
|name               |year_start|year_end|position|height|weight|birth_date       |college                              |
+-------------------+----------+--------+--------+------+------+-----------------+-------------------------------------+
|Alaa Abdelnaby     |1991      |1995    |F-C     |6-10  |240   |June 24, 1968    |Duke University                      |
|Zaid Abdul-Aziz    |1969      |1978    |C-F     |6-9   |235   |April 7, 1946    |Iowa State University                |
|Kareem Abdul-Jabbar|1970      |1989    |C       |7-2   |225   |April 1

Prebacivanje tipova podataka odredjenih kolona u odgovarajuce za analizu

In [0]:
df1.withColumn("birth_date",to_date(col("birth_date"),"Month-dd-yyyy"))
df1.printSchema()

root
 |-- name: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_end: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- college: string (nullable = true)



Uzimanje u obzir samo igrace cija karijera pocinje od 1990 godine pa nadalje

In [0]:
df1 = df1.select("*").where(col("year_start") > 1989)
df1.display()

name,year_start,year_end,position,height,weight,birth_date,college
Alaa Abdelnaby,1991,1995,F-C,6-10,240,"June 24, 1968",Duke University
Mahmoud Abdul-Rauf,1991,2001,G,6-1,162,"March 9, 1969",Louisiana State University
Tariq Abdul-Wahad,1998,2003,F,6-6,223,"November 3, 1974",San Jose State University
Shareef Abdur-Rahim,1997,2008,F,6-9,225,"December 11, 1976",University of California
Alex Abrines,2017,2018,G-F,6-6,190,"August 1, 1993",null
Alex Acker,2006,2009,G,6-5,185,"January 21, 1983",Pepperdine University
Quincy Acy,2013,2018,F,6-7,240,"October 6, 1990",Baylor University
Hassan Adams,2007,2009,G,6-4,220,"June 20, 1984",University of Arizona
Jordan Adams,2015,2016,G,6-5,209,"July 8, 1994","University of California, Los Angeles"
Steven Adams,2014,2018,C,7-0,255,"July 20, 1993",University of Pittsburgh


Cuvanje transformisanog dataseta u fajl

In [0]:
df1.write.csv("/tmp/PlayersDF", header = True)

## NBA Sezone sa statistikom svakog igraca za svaku sezonu od 1950 do 2021

U ovom datasetu se nalaze informacije o statistici pojedinacnog igraca za svaku NBA sezone, pocevsi od 1950. godine do 2021. godine

Tabela se sastoji iz sledecih kolona:
- **Player**: ime i prezime igraca
- **year**: trenutno posmatrana sezona statistike igraca
- **age**: trenutan broj godina igraca
- **G**: broj odigranih utakmica igraca 
- **MP**: ukupan broj odigranih minuta igraca 
- **FG**: ukupan broj postignutih koseva iz igre 
- **FGA**: ukupan broj pokusaja iz igre 
- **FG%**: procenat uspesnosti suta iz igre
- **2P**: ukupan broj postignutih koseva za dva poena 
- **2PA**: ukupan broj pokusaja za dva poena 
- **2P%**: procenat uspesnosti suta za dva poena
- **3P**: ukupan broj postignutih koseva za tri poena 
- **3PA**: ukupan broj pokusaja za tri poena
- **3P%**: procenat uspesnosti suta za tri poena
- **FT**: ukupan broj postignutih koseva sa slobodnih bacanja 
- **FTA**: ukupan broj pokusaja slobodnih bacanja
- **FT%**: procenat uspesnosti izvodjenja slobodnih bacanja
- **ORB**: broj ofanzivnih skokova 
- **DRB**: broj defanzivnih skokova 
- **TRB**: ukupan broj skokova igraca 
- **AST**: ukupan broj asistencija 
- **STL**: ukupan broj ukradenih lopti 
- **BLK**: ukupan broj blokiranih suteva
- **TOV**: ukupan broj izgubljenih lopti
- **PF**: ukupan broj nacinjenih faulova
- **PTS**: ukupan broj poena

izvor: https://www.kaggle.com/datasets/drgilermo/nba-players-stats?select=Seasons_Stats.csv // Kaggle

In [0]:
df2 = spark.read.option("header","true").csv("/FileStore/tables/Seasons_Stats.csv")


Kastovanje odredjenih kolona u odgovarajuce tipove promenljivih

In [0]:
df_season_stats = df2.withColumn("Year",col("Year").cast(IntegerType())) \
    .withColumnRenamed("_c0","index") \
    .withColumn("index",col("index").cast(IntegerType())) \
    .withColumn("Age",col("Age").cast(IntegerType())) \
    .withColumn("G",col("G").cast(IntegerType())) \
    .withColumn("GS",col("GS").cast(IntegerType())) \
    .withColumn("MP",col("MP").cast(IntegerType())) \
    .withColumn("FG",col("FG").cast(IntegerType())) \
    .withColumn("FGA",col("FGA").cast(DoubleType())) \
    .withColumn("FG%",col("FG%").cast(DoubleType())) \
    .withColumn("2P",col("2P").cast(IntegerType())) \
    .withColumn("2PA",col("2PA").cast(IntegerType())) \
    .withColumn("2P%",col("2P%").cast(DoubleType())) \
     .withColumn("3P",col("3P").cast(IntegerType())) \
    .withColumn("3PA",col("3PA").cast(IntegerType())) \
    .withColumn("3P%",col("3P%").cast(DoubleType())) \
    .withColumn("FT",col("FT").cast(IntegerType())) \
    .withColumn("FTA",col("FTA").cast(IntegerType())) \
    .withColumn("FT%",col("FT%").cast(FloatType())) \
    .withColumn("ORB",col("ORB").cast(IntegerType())) \
    .withColumn("DRB",col("DRB").cast(IntegerType())) \
    .withColumn("TRB",col("TRB").cast(IntegerType())) \
    .withColumn("AST",col("AST").cast(IntegerType())) \
    .withColumn("STL",col("STL").cast(IntegerType())) \
    .withColumn("BLK",col("BLK").cast(IntegerType())) \
    .withColumn("TOV",col("TOV").cast(IntegerType())) \
    .withColumn("PF",col("PF").cast(IntegerType())) \
    .withColumn("PTS",col("PTS").cast(IntegerType()))
    
df_season_stats.printSchema()
#display(df_season_stats)

root
 |-- index: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: string (nullable = true)
 |-- TS%: string (nullable = true)
 |-- 3PAr: string (nullable = true)
 |-- FTr: string (nullable = true)
 |-- ORB%: string (nullable = true)
 |-- DRB%: string (nullable = true)
 |-- TRB%: string (nullable = true)
 |-- AST%: string (nullable = true)
 |-- STL%: string (nullable = true)
 |-- BLK%: string (nullable = true)
 |-- TOV%: string (nullable = true)
 |-- USG%: string (nullable = true)
 |-- blanl: string (nullable = true)
 |-- OWS: string (nullable = true)
 |-- DWS: string (nullable = true)
 |-- WS: string (nullable = true)
 |-- WS/48: string (nullable = true)
 |-- blank2: string (nullable = true)
 |-- OBPM: string (nullable 

Selektovanje kolona koje ce se koristiti za analizu, kao i ogranicavanje data seta na podatke od 1990. godine

In [0]:
df_season_stats = df_season_stats.select(col("Player"), col("Year"), col("Age"), col("G"), col("GS"), 
                      col("MP"), col("FG"), col("FGA"), col("FG%"), col("2P"), 
                      col("2PA"), col("2P%"), col("3P"), col("3PA"), col("3P%"), col("FT"), col("FTA"), col("FT%"),
                      col("ORB"), col("DRB"), col("TRB"), col("AST"), col("STL"), 
                      col("BLK"), col("TOV"), col("PF"), col("PTS"), col("PER")) \
                      .where(col("Year") > 1989)
                                         
df_season_stats.display()

Player,Year,Age,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
Mark Acres,1990,27,80,50,1691,138,285.0,0.484,135,281,0.48,3,4,0.75,83,120,0.692,154,277,431,67,36,25,70,248,362,8.3
Michael Adams,1990,27,79,74,2690,398,989.0,0.402,240,557,0.431,158,432,0.366,267,314,0.85,49,176,225,495,121,3,141,133,1221,15.4
Mark Aguirre,1990,30,78,40,2005,438,898.0,0.488,407,805,0.506,31,93,0.333,192,254,0.756,117,188,305,145,34,19,121,201,1099,15.8
Danny Ainge,1990,30,75,68,2727,506,1154.0,0.438,398,865,0.46,108,289,0.374,222,267,0.831,69,257,326,453,113,18,185,238,1342,16.1
Mark Alarie,1990,26,82,10,1893,371,785.0,0.473,361,736,0.49,10,49,0.204,108,133,0.812,151,223,374,142,60,39,101,219,860,14.1
Steve Alford,1990,25,41,0,302,63,138.0,0.457,56,116,0.483,7,22,0.318,35,37,0.946,2,23,25,39,15,3,16,22,168,18.7
Randy Allen,1990,25,63,6,746,106,239.0,0.444,106,232,0.457,0,7,0.0,23,43,0.535,49,89,138,23,16,19,28,102,235,8.4
Greg Anderson,1990,25,60,28,1291,219,432.0,0.507,219,432,0.507,0,0,null,91,170,0.535,112,261,373,24,32,54,80,176,529,13
Nick Anderson,1990,22,81,9,1785,372,753.0,0.494,371,736,0.504,1,17,0.059,186,264,0.705,107,209,316,124,69,34,138,140,931,14.9
Richard Anderson,1990,29,54,2,604,88,211.0,0.417,51,111,0.459,37,100,0.37,18,23,0.783,33,94,127,55,20,9,26,64,231,13.3


Prebacivanje podataka za svakog igraca koji nije pokusao sut u bilo kojoj sezoni jer je bio setovan na null, a vrednost treba da bude 0

In [0]:
df_season_stats = df_season_stats.withColumn("2P%", when(col("2PA") == 0, 0).otherwise(col("2P%"))).withColumn("3P%", when(col("3PA") == 0, 0).otherwise(col("3P%")))\
                          .withColumn("FT%", when(col("FTA") == 0, 0).otherwise(col("FT%"))).withColumn("FG%", when(col("FGA") == 0, 0).otherwise(col("FT%")))

df_season_stats.select(col("Player"), col("2PA"), col("3PA"), col("FGA"), col("FTA")).display()


Player,2PA,3PA,FGA,FTA
Mark Acres,281,4,285.0,120
Michael Adams,557,432,989.0,314
Mark Aguirre,805,93,898.0,254
Danny Ainge,865,289,1154.0,267
Mark Alarie,736,49,785.0,133
Steve Alford,116,22,138.0,37
Randy Allen,232,7,239.0,43
Greg Anderson,432,0,432.0,170
Nick Anderson,736,17,753.0,264
Richard Anderson,111,100,211.0,23


Sredjivanje ispisa procenata suta iz svake kategorije i zaokruzivanje na prikaz u dve decimale

In [0]:
df_season_stats = df_season_stats.withColumn("FG%", round((col("FG%") * 100),2)).withColumn("2P%", round((col("2P%") * 100),2))\
                            .withColumn("3P%", round((col("3P%") * 100),2)).withColumn("FT%", round((col("FT%") * 100),2))

df_season_stats.select(col("2P"), col("2PA"), col("2P%"), 
                       col("3P"), col("3PA"), col("3P%"), 
                       col("FG"), col("FGA"), col("FG%"), 
                       col("FT"), col("FTA"), col("FT%"))\
                       .display()

2P,2PA,2P%,3P,3PA,3P%,FG,FGA,FG%,FT,FTA,FT%
135,281,48.0,3,4,75.0,138,285.0,69.2,83,120,69.2
240,557,43.1,158,432,36.6,398,989.0,85.0,267,314,85.0
407,805,50.6,31,93,33.3,438,898.0,75.6,192,254,75.6
398,865,46.0,108,289,37.4,506,1154.0,83.1,222,267,83.1
361,736,49.0,10,49,20.4,371,785.0,81.2,108,133,81.2
56,116,48.3,7,22,31.8,63,138.0,94.6,35,37,94.6
106,232,45.7,0,7,0.0,106,239.0,53.5,23,43,53.5
219,432,50.7,0,0,0.0,219,432.0,53.5,91,170,53.5
371,736,50.4,1,17,5.9,372,753.0,70.5,186,264,70.5
51,111,45.9,37,100,37.0,88,211.0,78.3,18,23,78.3


In [0]:
df_season_stats = df_season_stats.drop("GS")
df_season_stats.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- G: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- FG: integer (nullable = true)
 |-- FGA: double (nullable = true)
 |-- FG%: float (nullable = true)
 |-- 2P: integer (nullable = true)
 |-- 2PA: integer (nullable = true)
 |-- 2P%: double (nullable = true)
 |-- 3P: integer (nullable = true)
 |-- 3PA: integer (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FT: integer (nullable = true)
 |-- FTA: integer (nullable = true)
 |-- FT%: float (nullable = true)
 |-- ORB: integer (nullable = true)
 |-- DRB: integer (nullable = true)
 |-- TRB: integer (nullable = true)
 |-- AST: integer (nullable = true)
 |-- STL: integer (nullable = true)
 |-- BLK: integer (nullable = true)
 |-- TOV: integer (nullable = true)
 |-- PF: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- PER: string (nullable = true)



Cuvanje drugog dataseta u fajl za analizu

In [0]:
df_season_stats.write.csv("/tmp/season_statsDF", header = True)